In [ ]:
import requests
import re
from bs4 import BeautifulSoup

# Name에 소환사 이름 입력
Name = "엔터키봉인한남자"


hdr = {'Accept-Language': 'ko_KR,en;q=0.8', 'User-Agent': (
    'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.70 Mobile Safari/537.36')}
Container = {}
SummonerName = ""
Ranking = ""
Tier = []
LP = []
Wins = []
Losses = []
Ratio = []

# url 요청
url = 'https://www.op.gg/summoner/userName=' + Name
req = requests.get(url, headers=hdr)
html = req.text
soup = BeautifulSoup(html, 'html.parser')

# 여기서부터 크롤링
# 크롤링 후에는 Container 변수에 담아서 저장
for i in soup.select('div[class=SummonerName]'):
    SummonerName = i.text
Container['SummonerName'] = SummonerName

for i in soup.select('span[class=ranking]'):
    Ranking = i.text
Container['Ranking'] = Ranking

for j in soup.select('div[class=Tier]'):
    Tier.append(j.text.strip())
Container['Tier'] = Tier

for i in soup.select('div[class=LP]'):
    LP.append(i.text)
Container['LP'] = LP

for i in soup.select('span[class=Wins]'):
    if len(Wins) >= len(Tier):
        break
    Wins.append(i.text)
Container['Wins'] = Wins

for i in soup.select('span[class=Losses]'):
    if len(Losses) >= len(Tier):
        break
    Losses.append(i.text)
Container['Losses'] = Losses

for i in soup.select('span[class=Ratio]'):
    Ratio.append(i.text)
Container['Ratio'] = Ratio



# 모스트챔피언
most_champion = soup.select('div.MostChampionContent div.ChampionBox', limit = 3)
most_champion_list = []
  
for champ in most_champion:
    champ_info = champ.select_one('div.ChampionInfo')
    champ_name = champ_info.select_one('div.ChampionName').attrs['title']

    champ_minon_kill = re.sub('(\n|\t)', '', champ_info.select_one('div.ChampionMinionKill').text).split(' ')
    avr_CS = champ_minon_kill[1]
    minute_CS = re.sub('(\(|\))', '', champ_minon_kill[2])

    persnal_kda = champ.select_one('div.PersonalKDA')
    kda = persnal_kda.select_one('div.KDA span.KDA').text
    kill = persnal_kda.select_one('div.KDAEach span.Kill').text
    death = persnal_kda.select_one('div.KDAEach span.Death').text
    assist = persnal_kda.select_one('div.KDAEach span.Assist').text

    champ_winratio = re.sub('(\n|\t)', '', champ.select_one('div.Played div.WinRatio').text)
    champ_play_count = champ.select_one('div.Played div.Title').text.replace(' Played', '')

    most_champ = {
      "ChampionName": champ_name, 
      "AverageCS": float(avr_CS), 
      "CSperMinute": float(minute_CS), 
      "KDA": kda, 
      "Kill": float(kill), 
      "Death": float(death), 
      "Assist": float(assist), 
      "ChampionWinRatio": champ_winratio, 
      "ChampionPlayCount": champ_play_count
      }
    most_champion_list.append(most_champ)

Container['Most'] = most_champion_list

# 최근 20경기 평균 kill, death, assist 크롤링
recent20_kill = soup.select_one('div.KDA > span.Kill').text
recent20_death = soup.select_one('div.KDA > span.Death').text
recent20_assist = soup.select_one('div.KDA > span.Assist').text

Container['Recent_20_Kill_Average'] = recent20_kill
Container['Recent_20_Death_Average'] = recent20_death
Container['Recent_20_Assist_Average'] = recent20_assist



# 결과
Container